# Zombie Outbreak Simulation as a PDE Solver

## 1. The Original Differential Equations Model

The "When Zombies Attack!" paper by Munz et al. presents a system of ordinary differential equations (ODEs) modeling a zombie outbreak:

$$
\begin{align}
\frac{dS}{dt} &= \Pi - \beta SZ - \delta S \\
\frac{dI}{dt} &= \beta SZ - \rho I - \delta I \\
\frac{dZ}{dt} &= \rho I + \zeta R - \alpha SZ \\
\frac{dR}{dt} &= \delta S + \delta I + \alpha SZ - \zeta R
\end{align}
$$

Where:
- $S$: Susceptible humans
- $I$: Infected humans (not yet zombies)
- $Z$: Zombies
- $R$: Removed (dead) individuals
- $\Pi$: Birth rate
- $\beta$: Transmission rate when susceptible encounters zombie
- $\delta$: Natural death rate
- $\rho$: Rate at which infected become zombies
- $\alpha$: Rate at which zombies are destroyed by humans
- $\zeta$: Rate at which removed individuals resurrect as zombies

## 2. Extension to Partial Differential Equations

To incorporate spatial dynamics, we extend the model to partial differential equations (PDEs) by adding diffusion and advection terms:

$$
\begin{align}
\frac{\partial S}{\partial t} &= \Pi - \beta SZ - \delta S + D_S \nabla^2 S \\
\frac{\partial I}{\partial t} &= \beta SZ - \rho I - \delta I + D_I \nabla^2 I \\
\frac{\partial Z}{\partial t} &= \rho I + \zeta R - \alpha SZ + D_Z \nabla^2 Z \\
\frac{\partial R}{\partial t} &= \delta S + \delta I + \alpha SZ - \zeta R + D_R \nabla^2 R
\end{align}
$$

Where:
- $\nabla^2$ is the Laplacian operator representing diffusion
- $D_S, D_I, D_Z, D_R$ are diffusion coefficients for each population

## 3. Discretization of PDEs to Cellular Automata

### 3.1 Spatial Discretization

We discretize the continuous spatial domain into a grid of cells. For a 2D domain:
- Let $(i,j)$ represent the spatial indices of each cell
- Each cell has a size $\Delta x \times \Delta y$
- The physical position is $(x,y) = (i\Delta x, j\Delta y)$

### 3.2 Temporal Discretization

We discretize time into discrete steps:
- Let $n$ represent the temporal index
- Each time step has size $\Delta t$
- The time is $t = n\Delta t$

### 3.3 State Representation

In our cellular automaton:
- Each cell $(i,j)$ has one of 5 discrete states: Empty (0), Susceptible (1), Infected (2), Zombie (3), or Removed (4)
- Let $\phi_{i,j}^n$ represent the state of cell $(i,j)$ at time step $n$

## 4. From PDEs to Transition Rules

The PDE terms map to cellular automaton transition rules as follows:

### 4.1 Reaction Terms (Local Interactions)

The reaction terms in the PDEs (birth, infection, death, etc.) become probabilistic transition rules:

$$
\begin{align}
P(S \to I) &= \beta \cdot \frac{N_Z}{N_{total}} \cdot \Delta t \\
P(I \to Z) &= \rho \cdot \Delta t \\
P(Z \to R) &= \alpha \cdot \frac{N_S}{N_{total}} \cdot \Delta t \\
P(R \to Z) &= \zeta \cdot \Delta t \\
P(S \to R) = P(I \to R) &= \delta \cdot \Delta t \\
P(\emptyset \to S) &= \Pi \cdot \Delta t
\end{align}
$$

Where:
- $P(X \to Y)$ is the probability of transitioning from state $X$ to state $Y$
- $N_Z$ is the number of zombie neighbors
- $N_S$ is the number of susceptible neighbors
- $N_{total}$ is the total number of neighbors (8 in our Moore neighborhood)

### 4.2 Diffusion Terms

The diffusion terms $D \nabla^2 \phi$ are approximated by random movement of entities:

$$
P(\text{movement}) = \frac{D \cdot \Delta t}{(\Delta x)^2}
$$

For zombies, we implement this as a 50% chance to move to a random empty neighboring cell in each time step.

### 4.3 City-Specific Parameters

We extend the basic model by allowing parameters to vary spatially:
- Let $\beta_{i,j}, \rho_{i,j}, \alpha_{i,j}, \zeta_{i,j}, \delta_{i,j}, \Pi_{i,j}$ be the parameters at location $(i,j)$
- This allows for heterogeneous dynamics, with different "cities" having different properties

## 5. Numerical Analysis of the Method

### 5.1 Consistency

Our method is consistent with the original PDEs in the following sense:
- As $\Delta x, \Delta y, \Delta t \to 0$ and the number of states increases, our cellular automaton approximates the PDE solution
- The probabilistic rules converge to deterministic rates in the limit of large populations

### 5.2 Stability Conditions

For stability, we need to satisfy:
- $D \cdot \Delta t / (\Delta x)^2 \leq 0.5$ (the CFL condition for diffusion)
- All transition probabilities must be $\leq 1$

### 5.3 Convergence

By the Lax equivalence theorem, since our method is consistent and stable, it converges to the true solution of the PDE as the grid is refined.

## 6. Multi-scale Modeling

Our model implements multi-scale dynamics through:

### 6.1 Local Scale (Within Cities)

- Fine-grained interactions between individuals
- Local infection dynamics
- Short-range movement

### 6.2 Regional Scale (Between Cities)

- Long-distance zombie movement (1% probability)
- Parameter variations between regions
- Emerging patterns of regional infection spread

## 7. Implementation Details

### 7.1 Core Update Function

The core update function implements the discretized PDE solver:

```python
@numba.njit
def update_multi_city_grid(grid, param_grid, rng_seed=None):
    """Update the grid according to zombie infection rules"""
    ...
    # For each cell (i,j)
    for i in range(rows):
        for j in range(cols):
            # Get state and parameters
            state = grid[i, j]
            beta, rho, alpha, zeta, delta, pi = param_grid[i, j, :]
            
            # Count neighbors (approximating spatial gradients)
            zombie_neighbors = 0
            susceptible_neighbors = 0
            for di, dj in NEIGHBOR_OFFSETS:
                ni, nj = (i + di) % rows, (j + dj) % cols
                neighbor_state = grid[ni, nj]
                if neighbor_state == ZOMBIE:
                    zombie_neighbors += 1
                elif neighbor_state == SUSCEPTIBLE:
                    susceptible_neighbors += 1
            
            # Apply transition rules (discretized reaction terms)
            if state == EMPTY and r1 < pi:
                new_grid[i, j] = SUSCEPTIBLE
            elif state == SUSCEPTIBLE:
                if zombie_neighbors > 0 and r1 < beta * zombie_neighbors / 8:
                    new_grid[i, j] = INFECTED
                elif r2 < delta:
                    new_grid[i, j] = REMOVED
            # Additional transitions...
            
            # Zombie movement (diffusion term)
            elif state == ZOMBIE and r2 < 0.5:
                # Choose direction and move if target cell is empty
                ...
    ...
```

This function implements:
1. The reaction terms via state transitions
2. The diffusion terms via movement rules
3. The spatially varying parameters

### 7.2 Relation to Finite Difference Method

Our cellular automaton can be viewed as a stochastic finite difference method:
- The transition rules approximate the reaction terms
- The movement rules approximate the diffusion terms
- The neighborhood averaging approximates the Laplacian operator

## 8. Visualization and Analysis

The visualization components allow us to:
1. Observe spatial patterns that emerge from the PDE solutions
2. Track population dynamics over time
3. Compare outcomes between different parameter regimes
4. Analyze the spread rates and infection patterns

## 9. Mathematical Summary

Our cellular automaton zombie outbreak simulation is a numerical solver for the reaction-diffusion PDE system:

$$
\frac{\partial \phi_k}{\partial t} = R_k(\phi_1, \phi_2, \phi_3, \phi_4; \beta, \rho, \alpha, \zeta, \delta, \Pi) + D_k \nabla^2 \phi_k
$$

Where:
- $\phi_k$ represents the population density of state $k$
- $R_k$ represents the reaction terms (interactions between populations)
- $D_k \nabla^2 \phi_k$ represents the diffusion terms
- The parameters vary spatially to model different cities

The result is a sophisticated numerical method that captures both the mathematics of the original model and introduces realistic spatial heterogeneity.